In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import random

In [2]:
# Selenium 설정
options = webdriver.ChromeOptions()


In [3]:
# options.add_argument("--headless")  # 브라우저를 띄우지 않고 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


베너목록 = ['스포츠','자동차','전자','컴퓨터','휴대폰']

ii = int(input("인덱스를 입력하세요"))

# 알리 베스트셀러 url 접속
driver.get("https://www.aliexpress.com/gcp/300001064/aptaxwR82N?spm=a2g0o.best.testStatic.6.42dc423apfBW5D&disableNav=YES&pha_manifest=ssr&_immersiveMode=true")
# 창크기 최대로
driver.maximize_window()
# 명시적 대기 설정
wait = WebDriverWait(driver, 10)
time.sleep(5)
# 상단베너 검색
상단베너 = driver.find_elements(By.CSS_SELECTOR, 'div.rax-view-v2.tab_pc_content_item')
# 스포츠 및 엔터테인먼트 베너를 찾아서 클릭
for 베너 in 상단베너:
    if 베너목록[ii] in 베너.text:
        driver.execute_script("arguments[0].click();", 베너)
        break
# 추가 작업을 위해 잠시 대기
time.sleep(5)

# 화면크기를 줄여 많은 상품이 나오게 축소
driver.execute_script("document.body.style.zoom='75%'")
time.sleep(2)
body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.HOME)
body.send_keys(Keys.END)
time.sleep(1.9)
body.send_keys(Keys.END)
time.sleep(1.9)
body.send_keys(Keys.HOME)

KeyboardInterrupt: 

In [4]:
items = []
last_height = driver.execute_script("return document.body.scrollHeight")

# 50개의 항목이 존재할때 까지 페이지 스크롤을 내림(동적 페이지)
while len(items) < 50:
    #PgDn키 입력 (50개의 항목이 존재할 떄 까지)
    # body = driver.find_element(By.TAG_NAME, 'body')
    # body.send_keys(Keys.PAGE_DOWN)
    #스크롤내리고 제품이 뜨기까지 딜레이가 있어보여 5초간 대기
    # time.sleep(random.uniform(2, 4))
    # class="swiper-slide swiper-slide-active" 요소 찾기
    swiper_slide = driver.find_element(By.CLASS_NAME, "swiper-slide.swiper-slide-active")
    # 하위의 class="productContainer" 요소들 찾기
    items = swiper_slide.find_elements(By.CLASS_NAME, "productContainer")
    # 새로운 높이를 계산하여 이전 높이와 비교
    new_height = driver.execute_script("return document.body.scrollHeight")
    # 높이가 변하지 않으면 더 이상 항목이 로드되지 않음을 의미
    if new_height == last_height:
        print("더 이상 항목이 로드되지 않음")
        break
    
    last_height = new_height

print(f"총 로드된 항목 수: {len(items)}")
# df = pd.DataFrame({"상품명": [], '가격': [], '평점': [], '댓글': [], '별점': [], '링크': []})

더 이상 항목이 로드되지 않음
총 로드된 항목 수: 74


In [5]:
def 한글포함유무(text):
    for char in text:
        if '가' <= char <= '힣':
            return True
    return False
            
상품명_total = []
평점_total = []
가격_total = []
링크_total = []
댓글_total = []
별점_total = []
for ii, item in enumerate(items):
    if ii>50:
        break
    # 요소가 화면에 보이도록 스크롤
    driver.execute_script("arguments[0].scrollIntoView();", item)
    # 자바스크립트를 사용하여 클릭
    driver.execute_script("arguments[0].click();", item)
    # 마지막 탭으로 이동
    last_tab = driver.window_handles[-1]
    driver.switch_to.window(window_name=last_tab)

    # 상품정보 파싱
    상품명 = [wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".title--wrap--Ms9Zv4A > h1"))).text]
    평점 = [wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.reviewer--wrap--sPGWrNq > strong"))).text.strip()]
    가격 = [wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product-price-current"))).text[1:]]
    링크 = [driver.current_url]  # 상품 링크 수집
    
    time.sleep(5)
    # 댓글 및 별점수집
    한글댓글 = []
    한글댓글의별점 = []
    
    # 화면크기를 줄여 많은 상품이 나오게 축소
    # driver.execute_script("document.body.style.zoom='25%'")    
    try:
            #end키 입력
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.END)
            
            # 댓글 더보기 버튼 클릭
            더보기_버튼 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pdp-body-top-left .review--wrap--yhZbZox .ae-evaluation-list .comet-v2-btn.comet-v2-btn-slim.comet-v2-btn-large.comet-v2-btn-important")))
            # 더보기 버튼이 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", 더보기_버튼)
            driver.execute_script("arguments[0].click();", 더보기_버튼)
            time.sleep(random.uniform(2, 2.5))
            
            # 원래 번역 표시 버튼 클릭
            try:
                원래번역표시 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ae-all-list-content .ae-filter-select-box .ae-filter-select-translate')))
                원래번역표시.click()
                time.sleep(2)  # 버튼 클릭 후 잠시 대기
            except Exception as e:
                print(f"원래 번역 표시 버튼 클릭 실패: {e}")
    except:
            pass

    추가댓글수집여부카운트 = 0
    while len(한글댓글) < 100:
        try:        
            # 팝업 창의 스크롤 영역 식별
            popup_scroll_area = driver.find_element(By.CSS_SELECTOR, "div.ae-all-list-content")
                
            # 팝업 창 스크롤하여 댓글 추가 수집
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", popup_scroll_area)
            time.sleep(2)
        except:
            pass
        

        한글댓글수 = len(한글댓글)
        추가댓글수집여부 = 1
        
        # 댓글 및 별점 파싱
        댓글_elements = driver.find_elements(By.CSS_SELECTOR, ".ae-evaluateList-card .ae-evaluateList-card-content")
        별점_elements = driver.find_elements(By.CSS_SELECTOR, ".ae-evaluateList-card .ae-stars-box")
        
        if len(댓글_elements) != len(별점_elements):
            print(f'경고_댓글 개수와 별점개수가 다릅니다_상품명 : {상품명}')
            break
            
        추가된_댓글_수 = 0
        for 댓글_element,별점_element in zip(댓글_elements,별점_elements):
            댓글_1 = 댓글_element.text
            star_images = 별점_element.find_elements(By.CSS_SELECTOR, "img.ae-stars")
            star_count = sum(1 for img in star_images if "S3a90fde0c9cc4b8c8958a6ee7553e26bf/42x42.png" in img.get_attribute("src"))
            if 한글포함유무(댓글_1) and 댓글_1 not in 한글댓글:
                한글댓글.append(댓글_1)
                한글댓글의별점.append(star_count)
                추가된_댓글_수 += 1
                
                if len(한글댓글) >= 100:
                    print("댓글 100개 수집완료, 다음 상품으로 넘어갑니다.")
                    break
            
        # page_down 5회 이상하여 새로운 글이 없을 때,
        if 추가된_댓글_수 > 0:
            추가댓글수집여부카운트 = 0
        else:
            추가댓글수집여부카운트 += 1
            
        if 추가댓글수집여부카운트 > 3:
            print("스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.")
            break
    
    
    상품명_total = 상품명_total+상품명
    평점_total = 평점_total+평점
    가격_total = 가격_total+가격
    링크_total = 링크_total+링크
    댓글_total = 댓글_total+[한글댓글]
    별점_total = 별점_total+[한글댓글의별점]
    
    # 현재 탭 닫기
    driver.close()

    # 메인탭으로 이동
    main_tab = driver.window_handles[0]
    driver.switch_to.window(window_name=main_tab)

data = {
    '상품명':상품명_total,
    '평점':평점_total,
    '가격':가격_total,
    '댓글':댓글_total,
    '별점':별점_total,
    '링크':링크_total
        }
df = pd.DataFrame(data)


# driver 종료
driver.quit()

스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
댓글 100개 수집완료, 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 없으므로 다음 상품으로 넘어갑니다.
스크롤 5회 이상 내려 한글 댓글을 불러 올 수 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     51 non-null     object
 1   평점      51 non-null     object
 2   가격      51 non-null     object
 3   댓글      51 non-null     object
 4   별점      51 non-null     object
 5   링크      51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [7]:
df.to_csv('data_2/'+베너목록[aa]+'.csv',index=False,encoding='utf-8-sig')